# Testing Notebook

Execute a Testplan with Azure Data Studio, execute a Tesplan with Pester

## Pre Requisites

### Start container mssqlsrv1

In [1]:
# -- Start mssqlsrv1
$null = docker start mssqlsrv1
docker ps -a --filter "name=mssqlsrv1"

CONTAINER ID   IMAGE               COMMAND                  CREATED       STATUS                  PORTS                                       NAMES
e7ff4a53d0dd   mssql:2019-latest   "/opt/mssql/bin/perm…"   3 weeks ago   Up Less than a second   0.0.0.0:8433->1433/tcp, :::8433->1433/tcp   mssqlsrv1


### Create a TestingDB on MSSQL Server

In [ ]:
# -- Create new database 'TestingDB'
$database = 'TestingDB'
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred
$dbexists = Get-DbaDatabase -SqlInstance $mssqlsrv  -Database $database
if($dbexists){
    "$database already exists"
}else{
    New-DbaDatabase -SqlInstance $mssqlsrv -Name $database
}

### Create new Testing Table

In [ ]:
# -- Create new table 'Test_Case1'
$table    = 'Test_Case1'
$database = 'TestingDB'
$Columns  =  @()
$Columns  += @{
    Name      = 'ID'
    Type      = 'int'
    Identity  = $true
    Nullable  = $false
}
$Columns  += @{
    Name      = 'TimeStamp'
    Type      = 'datetime'
    Nullable  =  $false
    Default   =  'GETDATE()'
}
$Columns  += @{
    Name      = 'ComputerName'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Testcase'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'HowToTest'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'ExpectedResult'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'ReceivedResult'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Status'
    Type      = 'Nvarchar'
    MaxLength = 10
    Nullable  =  $true
}
$tblexists = Get-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Table $table
if($tblexists){
    "$tblexists already exists"
}else{
    New-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Name $table -ColumnMap $Columns | Select SqlInstance, Database, Name
}

## Testplan Azure Data Studio

### Test preparation

In [2]:
# -- Connect to MSSQL Server
$database = 'TestingDB'
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($mssqlsrv)){
    if([String]::IsNullOrEmpty($sqlcred)){
        $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
        $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
    }
    $mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred
    $mssqlsrv
}else{
    "already connected to $database"
}

already connected to TestingDB


In [ ]:
# -- Start containers
$container = 'mongodb1'
$result = docker ps -a --filter "name=$container"
if([String]::IsNullOrEmpty($result)){
    docker start $container
}else{
    "$container already running"
}

Test Computer

In [ ]:
$ComputerName = 'TESTSRV232789'

### Test image on local docker repository

In [ ]:
# -- Test 1 docker image
$image = 'mongo'
$result = docker image ls $image 
$string = $result[1].Trim() -split '\s+'

$Testcase       = "Test is Image $image on local docker repository"
$HowToTest      = "docker image ls $image"
$ExpectedResult = "Repository, Tag, Image ID, Date created, Size of the Image"
$ReceivedResult = $string -join ", "
$PassFail       = if($ReceivedResult -match $image){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,ComputerName,Testcase,HowToTest,ExpectedResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($ComputerName)','$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

### Test the status of the docker container

In [ ]:
# -- Test 2 docker container
$container = 'mongodb1'
$result = docker ps -a --filter "name=$container"
$string = $result[1].Trim() -split '\s+'

$Testcase       = "Test is container $container on local docker repository"
$HowToTest      = "docker ps -a --filter ""name=$container"""
$ExpectedResult = "Container ID, Image, Command, Date created, Status, Ports, Names"
$ReceivedResult = $string -join ", "
$PassFail       = if($ReceivedResult -match $container){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,ComputerName,Testcase,HowToTest,ExpectedResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($ComputerName)','$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

### Test the MongoDB port-mapping

In [ ]:
# -- Test 3 mongodb access
$container = 'mongodb1'
$ports = docker inspect --format='{{.NetworkSettings.Ports}}' $container
$result = Invoke-WebRequest -Uri 'http://localhost:27017'

$Testcase       = "Test is $container is accessible over http"
$HowToTest      = "Invoke-WebRequest -Uri ""http://localhost:27017"""
$ExpectedResult = "StatusCode 200, StatusDescription OK"
$ReceivedResult = "$($result.StatusCode), $($result.StatusDescription), $($result.Content)"
$PassFail       = if($result.StatusCode -eq 200){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,ComputerName,Testcase,HowToTest,ExpectedResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($ComputerName)','$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

### Test the MongoDB database 

In [ ]:
# -- Test 4 mongodb database
if(-not(Get-InstalledModule -Name Mdbc)){
    Install-Module -Name Mdbc
}
if(-not(Get-Module -Name Mdbc)){
    Import-Module -Name Mdbc
}

$mongodb = 'admin'
$mongo_databases = Connect-Mdbc mongodb://localhost:27017 | Get-MdbcDatabase
$result = $mongo_databases.DatabaseNamespace
$dbs = $null
foreach($item in $result.DatabaseName){
    $dbs = $item + ", " + $dbs
}

$Testcase       = "Test is $mongodb database on $container is available"
$HowToTest      = "Connect-Mdbc mongodb://localhost:27017 | Get-MdbcDatabase"
$ExpectedResult = "DatabaseNamespace $mongodb"
$ReceivedResult = $dbs.TrimEnd(', ')
$PassFail       = if($result -contains $mongodb){'Pass'}else{'Fail'}
$PassFail

# -- INSERT INTO [TestingDB].[dbo].[Test_Case1]
$SqlQuery   = @"
INSERT INTO [$database].[dbo].[$table] (
    TimeStamp,ComputerName,Testcase,HowToTest,ExpectedResult,ReceivedResult,Status
)
VALUES (
    CURRENT_TIMESTAMP,'$($ComputerName)','$($Testcase)','$($HowtoTest)','$($ExpectetResult)','$($ReceivedResult)','$($PassFail)'
)
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -verbose

### Inspect container

In [ ]:
# -- docker inspect mongodb1
$container = @('mongodb1')
foreach($item in $container){
    [PSCustomObject]@{
        Hostname  = docker inspect --format='{{.Config.Hostname}}' $item 
        IPAddress = docker inspect --format='{{.NetworkSettings.Networks.custom.IPAddress}}' $item 
        Path      = docker inspect --format='{{.Path}}' $item 
        Ports     = docker inspect --format='{{.NetworkSettings.Ports}}' $item 
        ExposedPorts = docker inspect --format='{{.Config.ExposedPorts}}' $item 
    }
}
#docker inspect mongodb1

## Import from CSV

Import data from CSV-file into the database

In [ ]:
# -- Import CSV
$database = 'TestingDB'
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred -Database $database
$Downloads = Join-Path "$($env:USERPROFILE)" -ChildPath 'Downloads'
Import-DbaCsv -Path (Join-Path $Downloads -ChildPath 'Test_MongoDB.csv') -SqlInstance $mssqlsrv -Database $database -NullValue "NULL" -AutoCreateTable

## Pester Tests

The preparations are in the PetserTest-file in ./lib/MongoDB.Tests.ps1

### Execute Pester Tests

In [ ]:
$PesterResult = Invoke-Pester -PassThru -Path 'D:\DevOps\github.com\Jupyter-Notebooks\lib'

### Get Test results as PSObject

In [ ]:
$AllPesterResults = @()
$ComputerName = 'TESTSRV232789'
$ColumnMap = @(
    @{l='ID';e={New-Guid}},
    @{l='TimeStamp';e={"$($_.ExecutedAt.ToString())"}},
    @{l='Computer';e={$ComputerName}},
    @{l='Testcase';e={"$($_.ExpandedPath.ToString().split('.')[0])"}},
    @{l='HowToTest';e={"$($_.ScriptBlock.ToString().Trim())"}},
    @{l='ExpectedResult';e={"$($_.ExpandedName)"}},
    @{l='ReceivedResult';e={"$($_.Result)"}},
    @{l='Status';e={"$($_.Passed.ToString())"}}
)
$AllPesterResults += $PesterResult.Passed | Select-Object $ColumnMap
$ColumnMap = @(
    @{l='ID';e={New-Guid}},
    @{l='TimeStamp';e={"$($_.ExecutedAt.ToString())"}},
    @{l='Computer';e={$ComputerName}},
    @{l='Testcase';e={"$($_.ExpandedPath.ToString().split('.')[0])"}},
    @{l='HowToTest';e={"$($_.ScriptBlock.ToString().Trim())"}},
    @{l='ExpectedResult';e={"$($_.ExpandedName.ToString())"}},
    @{l='ReceivedResult';e={"$($_.ErrorRecord.DisplayErrorMessage.ToString())"}},
    @{l='Status';e={"$($_.Passed.ToString())"}}
)
$AllPesterResults += $PesterResult.Failed | Select-Object $ColumnMap
$AllPesterResults

### Save Test results as CSV-file

In [ ]:
# -- csv
$Downloads = Join-Path "$($env:USERPROFILE)" -ChildPath 'Downloads'
$AllPesterResults | ConvertTo-Csv -Delimiter ',' -NoTypeInformation | Set-Content -Path (Join-Path $Downloads -ChildPath 'PesterTest.csv') -Encoding utf8 -PassThru

### Save Test results as JSON-file

In [ ]:
# -- json
$Downloads = Join-Path "$($env:USERPROFILE)" -ChildPath 'Downloads'
$AllPesterResults | ConvertTo-JSON -Compress | Set-Content -Path (Join-Path $Downloads -ChildPath 'PesterTest.json') -Encoding utf8 -PassThru

### Import Test results into the database

In [ ]:
# -- Write-DbaDbTableData
$table = 'TestMongoDB'
$dataset = $AllPesterResults | ConvertTo-DbaDataTable
$dataset | Write-DbaDbTableData -SqlInstance $mssqlsrv -Database $database -Table $table -AutoCreateTable -Verbose #-Truncate

## Get Test results from database

In [ ]:
$tablecol = @('Test_Case1','TestMongoDB')
# -- SELECT * FROM [TestingDB].[dbo].[Test_Case1]
$result = foreach($table in $tablecol){
    $SqlQuery  = "SELECT * FROM [$database].[dbo].[$table]"
    Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $database -Query $SqlQuery -WarningAction SilentlyContinue | Select @{l='Table';e={"$($table)"}},ID,TimeStamp,ComputerName,Testcase,Status
}
$result | Format-Table -GroupBy Table 

# Backup, Restore

In [7]:
Backup-DbaDatabase -SqlInstance $mssqlsrv -Path '/var/opt/mssql/backup' -Database $database -Type Full -IgnoreFileChecks | fl *


BackupComplete        : True
BackupFile            : TestingDB_202106020915.bak
BackupFilesCount      : 1
BackupFolder          : /var/opt/mssql/backup
BackupPath            : /var/opt/mssql/backup\TestingDB_202106020915.bak
DatabaseName          : TestingDB
Notes                 : 
Script                : BACKUP DATABASE [TestingDB] TO  DISK = N'/var/opt/mssql
                        /backup\TestingDB_202106020915.bak' WITH NOFORMAT, NOIN
                        IT, NOSKIP, REWIND, NOUNLOAD,  STATS = 1
Verified              : False
ComputerName          : localhost
InstanceName          : MSSQLSERVER
SqlInstance           : mssqlsrv1
AvailabilityGroupName : 
Database              : TestingDB
UserName              : sa
Start                 : 06/02/2021 09:15:33
End                   : 06/02/2021 09:15:33
Duration              : 00:00:00
Path                  : {/var/opt/mssql/backup\TestingDB_202106020915.bak}
TotalSize             : 3.02 MB
CompressedBackupSize  : 3.02 MB
Compressio

In [10]:
$BackupPath = '/var/opt/mssql/backup/TestingDB_202106020915.bak'
Restore-DbaDatabase -SqlInstance $mssqlsrv -Path $BackupPath -DatabaseName $database -WithReplace | fl *

                                                                                 Updating                                                                           Progress                                                                        [                                                                    ]                                                                                          Scanning Restore headers: 0/1                                                                                                                               